In [1]:
import os
print(os.listdir("input"))

['small1.csv', 'small2.csv']


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.sparse as scs # sparse matrix construction 
import scipy.linalg as scl # linear algebra algorithms
import scipy.optimize as sco # for minimization use
import matplotlib.pylab as plt # for visualization

def fixed_constraints(N=9):
    rowC = np.zeros(N)
    rowC[0] =1
    rowR = np.zeros(N)
    rowR[0] =1
    row = scl.toeplitz(rowC, rowR)
    ROW = np.kron(row, np.kron(np.ones((1,N)), np.eye(N)))
    
    colR = np.kron(np.ones((1,N)), rowC)
    col  = scl.toeplitz(rowC, colR)
    COL  = np.kron(col, np.eye(N))
    
    M = int(np.sqrt(N))
    boxC = np.zeros(M)
    boxC[0]=1
    boxR = np.kron(np.ones((1, M)), boxC) 
    box = scl.toeplitz(boxC, boxR)
    box = np.kron(np.eye(M), box)
    BOX = np.kron(box, np.block([np.eye(N), np.eye(N) ,np.eye(N)]))
    
    cell = np.eye(N**2)
    CELL = np.kron(cell, np.ones((1,N)))
    
    return scs.csr_matrix(np.block([[ROW],[COL],[BOX],[CELL]]))


# For the constraint from clues, we extract the nonzeros from the quiz string.
def clue_constraint(input_quiz, N=9):
    m = np.reshape([int(c) for c in input_quiz], (N,N))
    r, c = np.where(m.T)
    v = np.array([m[c[d],r[d]] for d in range(len(r))])
    
    table = N * c + r
    table = np.block([[table],[v-1]])
    
    # it is faster to use lil_matrix when changing the sparse structure.
    CLUE = scs.lil_matrix((len(table.T), N**3))
    for i in range(len(table.T)):
        CLUE[i,table[0,i]*N + table[1,i]] = 1
    # change back to csr_matrix.
    CLUE = CLUE.tocsr() 
    
    return CLUE

In [3]:
import time

# We test the following algoritm on small data set.
data = pd.read_csv("input/small2.csv") 

corr_cnt = 0
start = time.time()

random_seed = 42
np.random.seed(random_seed)

if len(data) > 1000:
    samples = np.random.choice(len(data), 1000)
else:
    samples = range(len(data))

for i in range(len(samples)):
    quiz = data["quizzes"][samples[i]]
    solu = data["solutions"][samples[i]]
    #print(quiz)
    def solver(quiz, solu):
    
        A0 = fixed_constraints()
        A1 = clue_constraint(quiz)

        # Formulate the matrix A and vector B (B is all ones).
        A = scs.vstack((A0,A1))
        A = A.toarray()
        B = np.ones(A.shape[0])

        u, s, vh = np.linalg.svd(A, full_matrices=False)
        K = np.sum(s > 1e-12)
        S = np.block([np.diag(s[:K]), np.zeros((K, A.shape[0]-K))])
        A = S@vh
        B = u.T@B
        B = B[:K]

        c = np.block([ np.ones(A.shape[1]), np.ones(A.shape[1]) ])
        G = np.block([[-np.eye(A.shape[1]), np.zeros((A.shape[1], A.shape[1]))],\
                             [np.zeros((A.shape[1], A.shape[1])), -np.eye(A.shape[1])]])
        h = np.zeros(A.shape[1]*2)
        H = np.block([A, -A])
        b = B

        ret = sco.linprog(c, G, h, H, b, method='interior-point', options={'tol':1e-6})
        x = ret.x[:A.shape[1]] - ret.x[A.shape[1]:]


        z = np.reshape(x, (81, 9))
    
        return z
    z = solver(quiz, solu)
    #print(z)
    res = np.linalg.norm(np.reshape(np.array([np.argmax(d)+1 for d in z]), (9,9) ) \
                      - np.reshape([int(c) for c in solu], (9,9)), np.inf) 
    
    if res>0:
        a = np.array([np.argmax(d)+1 for d in z])- np.array([int(c) for c in solu])
        #print(a)
        #print(z)
        final = np.reshape(np.array([np.argmax(d)+1 for d in z]), -1)
        for j in range(a.shape[0]):
            if a[j] != 0:
                
                #print(final)
                final[j] = 0
        #print("here")
        #print(final)
     
        new_x = solver(final, solu)
        print(np.reshape(np.array([np.argmax(d)+1 for d in new_x]), -1))
        if np.linalg.norm(np.reshape(np.array([np.argmax(d)+1 for d in new_x]), (9,9) ) \
                       - np.reshape([int(c) for c in solu], (9,9)), np.inf) >0:
                pass
        else:
            corr_cnt+=1

    else:
        #print("CORRECT")
        print(np.reshape(np.array([np.argmax(d)+1 for d in z]), -1))
        corr_cnt += 1
    print(solu)
    
    if (i+1) % 20 == 0:
        end = time.time()
        print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

end = time.time()
print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )


[9 7 3 5 6 2 8 4 1 6 2 5 1 4 8 7 9 3 4 8 1 3 9 7 2 5 6 8 5 9 7 1 4 6 3 2 7
 1 4 2 3 6 5 8 9 3 6 2 8 5 9 1 7 4 1 9 8 6 7 3 4 2 5 2 4 6 9 8 5 3 1 7 5 3
 7 4 2 1 9 6 8]
973562841625148793481397256859714632714236589362859174198673425246985317537421968
[5 9 3 4 2 6 8 7 1 4 6 2 7 8 1 3 5 9 1 8 7 5 3 9 4 2 6 3 4 1 6 9 5 2 8 7 8
 5 6 3 7 2 1 9 4 7 2 9 8 1 4 5 6 3 2 7 4 9 5 3 6 1 8 6 1 8 2 4 7 9 3 5 9 3
 5 1 6 8 7 4 2]
593426871462781359187539426341695287856372194729814563274953618618247935935168742
[6 7 4 9 2 8 5 1 3 1 5 3 4 6 7 8 2 9 2 9 8 3 5 1 7 4 6 5 3 1 7 9 2 4 6 8 8
 2 6 5 1 4 9 3 7 9 4 7 8 3 6 2 5 1 7 6 5 2 8 3 1 9 4 3 8 2 1 4 9 6 7 5 4 1
 9 6 7 5 3 8 2]
674928513153467829298351746531792468826514937947836251765283194382149675419675382
[8 1 3 7 5 9 4 6 2 2 5 9 6 1 4 8 7 3 4 7 6 3 8 2 1 5 9 7 8 2 5 3 6 9 4 1 9
 4 1 2 7 8 6 3 5 6 3 5 9 4 1 7 2 8 3 6 8 4 9 5 2 1 7 5 9 4 1 2 7 3 8 6 1 2
 7 8 6 3 5 9 4]
813759462259614873476382159782536941941278635635941728368495217594127386127863594
[9 7 2 1

[8 5 2 4 6 3 9 7 1 1 7 9 2 5 8 6 4 3 3 4 6 9 7 1 2 8 5 9 1 8 7 4 6 5 3 2 7
 6 4 3 2 5 8 1 9 2 3 5 1 8 9 4 6 7 6 8 3 5 9 7 1 2 4 4 9 1 8 3 2 7 5 6 5 2
 7 6 1 4 3 9 8]
852463971179258643346971285918746532764325819235189467683597124491832756527614398
[7 8 1 3 4 5 6 2 9 2 9 6 8 7 1 3 5 4 3 4 5 2 6 9 1 8 7 6 3 7 4 1 2 8 9 5 9
 5 8 7 3 6 4 1 2 1 2 4 9 5 8 7 3 6 5 6 3 1 9 7 2 4 8 4 7 2 5 8 3 9 6 1 8 1
 9 6 2 4 5 7 3]
781345629296871354345269187637412895958736412124958736563197248472583961819624573
[4 7 9 2 3 6 8 1 5 6 8 3 5 1 4 9 7 2 5 2 1 7 9 8 6 3 4 2 4 6 3 5 7 1 9 8 1
 9 8 4 6 2 3 5 7 7 3 5 1 8 9 4 2 6 9 5 4 6 2 3 7 8 1 3 6 2 8 7 1 5 4 9 8 1
 7 9 4 5 2 6 3]
479236815683514972521798634246357198198462357735189426954623781362871549817945263
[7 1 9 5 6 2 8 4 3 2 6 5 4 3 8 1 7 9 8 4 3 9 1 7 6 5 2 1 9 8 2 7 4 5 3 6 4
 7 2 6 5 3 9 1 8 3 5 6 8 9 1 7 2 4 9 3 7 1 4 6 2 8 5 6 2 4 7 8 5 3 9 1 5 8
 1 3 2 9 4 6 7]
719562843265438179843917652198274536472653918356891724937146285624785391581329467
[8 4 7 5

[1 6 5 8 4 2 9 7 3 2 8 4 3 9 7 6 5 1 7 3 9 5 6 1 4 8 2 6 7 3 9 8 4 1 2 5 8
 4 2 1 7 5 3 6 9 5 9 1 6 2 3 8 4 7 3 5 7 4 1 6 2 9 8 4 2 8 7 3 9 5 1 6 9 1
 6 2 5 8 7 3 4]
165842973284397651739561482673984125842175369591623847357416298428739516916258734
[9 4 5 2 1 3 6 7 8 8 7 6 9 4 5 3 2 1 2 3 1 6 7 8 4 9 5 3 1 9 8 6 7 5 4 2 4
 5 2 3 9 1 7 8 6 6 8 7 5 2 4 1 3 9 1 9 8 4 3 6 2 5 7 5 6 3 7 8 2 9 1 4 7 2
 4 1 5 9 8 6 3]
945213678876945321231678495319867542452391786687524139198436257563782914724159863
[5 8 2 9 4 6 7 3 1 9 3 4 1 8 7 6 2 5 6 7 1 2 3 5 8 4 9 1 6 8 4 9 2 3 5 7 3
 4 5 7 6 8 9 1 2 7 2 9 5 1 3 4 8 6 4 5 6 8 7 1 2 9 3 2 9 3 6 5 4 1 7 8 8 1
 7 3 2 9 5 6 4]
582946731934187625671235849168492357345768912729513486456871293293654178817329564
[2 7 6 3 5 8 4 1 9 1 9 3 6 7 4 2 8 5 8 5 4 9 1 2 7 6 3 5 3 7 1 4 6 8 9 2 9
 6 8 7 2 3 1 5 4 4 2 1 8 9 5 3 7 6 6 8 9 4 3 1 5 2 7 3 1 2 5 6 7 9 4 8 7 4
 5 2 8 9 6 3 1]
276358419193674285854912763537146892968723154421895376689431527312567948745289631
[5 9 4 7

[1 9 8 7 5 6 2 3 4 2 7 6 4 3 8 9 5 1 4 3 5 1 2 9 7 8 6 9 6 2 3 8 4 5 1 7 7
 5 1 9 6 2 3 4 8 8 4 3 5 1 7 6 2 9 5 2 7 6 4 1 8 9 3 6 8 4 2 9 3 1 7 5 3 1
 9 8 7 5 4 6 2]
198756234276438951435129786962384517751962348843517629527641893684293175319875462
Aver Time:   0.67 secs. Success rate: 100 / 100 
[1 9 8 5 7 6 4 3 2 7 5 6 4 2 3 9 8 1 4 2 3 8 9 1 6 5 7 6 7 1 9 3 5 2 4 8 8
 4 2 6 1 7 5 9 3 9 3 5 2 8 4 7 1 6 2 8 7 3 5 9 1 6 4 5 1 4 7 6 8 3 2 9 3 6
 9 1 4 2 8 7 5]
198576432756423981423891657671935248842617593935284716287359164514768329369142875
[8 5 6 2 9 7 1 4 3 3 1 9 4 6 8 7 5 2 7 4 2 3 1 5 9 8 6 5 6 1 7 8 4 3 2 9 4
 8 7 9 2 3 5 6 1 2 9 3 6 5 1 4 7 8 1 2 4 5 3 6 8 9 7 9 7 8 1 4 2 6 3 5 6 3
 5 8 7 9 2 1 4]
856297143319468752742315986561784329487923561293651478124536897978142635635879214
[4 7 5 6 3 8 1 9 2 3 6 8 9 2 1 7 5 4 2 1 9 4 7 5 3 8 6 5 2 6 1 4 7 9 3 8 9
 8 3 5 6 2 4 7 1 1 4 7 3 8 9 6 2 5 8 3 1 2 9 6 5 4 7 6 9 2 7 5 4 8 1 3 7 5
 4 8 1 3 2 6 9]
47563819236892175421947538652614793898356

[4 9 6 2 3 1 8 5 7 7 5 2 4 6 8 9 3 1 3 1 8 5 7 9 4 2 6 6 4 1 3 8 7 2 9 5 9
 8 3 6 2 5 7 1 4 2 7 5 1 9 4 6 8 3 1 2 7 9 5 6 3 4 8 8 3 4 7 1 2 5 6 9 5 6
 9 8 4 3 1 7 2]
496231857752468931318579426641387295983625714275194683127956348834712569569843172
[8 4 7 5 9 6 2 3 1 2 5 3 7 4 1 6 8 9 1 6 9 2 3 8 4 5 7 9 8 2 1 6 4 5 7 3 4
 1 6 3 7 5 9 2 8 7 3 5 9 8 2 1 6 4 6 7 1 4 2 3 8 9 5 5 9 8 6 1 7 3 4 2 3 2
 4 8 5 9 7 1 6]
847596231253741689169238457982164573416375928735982164671423895598617342324859716
[1 2 8 4 5 7 9 6 3 6 3 9 1 2 8 4 7 5 4 7 5 9 6 3 8 2 1 8 6 2 7 1 9 3 5 4 3
 9 7 2 4 5 1 8 6 5 4 1 3 8 6 7 9 2 7 5 4 6 9 1 2 3 8 2 8 3 5 7 4 6 1 9 9 1
 6 8 3 2 5 4 7]
128457963639128475475963821862719354397245186541386792754691238283574619916832547
[6 1 8 7 3 5 2 4 9 5 4 7 2 9 6 8 1 3 3 9 2 1 8 4 6 7 5 7 2 1 5 6 8 9 3 4 4
 5 6 9 1 3 7 2 8 9 8 3 4 7 2 1 5 6 1 6 4 8 5 7 3 9 2 2 3 9 6 4 1 5 8 7 8 7
 5 3 2 9 4 6 1]
618735249547296813392184675721568934456913728983472156164857392239641587875329461
[4 7 2 8

...

[3 7 1 6 9 5 2 4 8 6 5 9 2 8 4 3 1 7 2 4 8 1 3 7 6 5 9 9 8 3 5 1 2 4 7 6 5
 2 4 8 7 6 1 9 3 1 6 7 9 4 3 5 8 2 7 1 5 3 2 8 9 6 4 4 9 2 7 6 1 8 3 5 8 3
 6 4 5 9 7 2 1]
371695248659284317248137659983512476524876193167943582715328964492761835836459721
[5 8 2 9 7 3 6 1 4 1 6 4 5 8 2 3 7 9 3 9 7 6 1 4 8 5 2 7 3 5 1 4 9 2 8 6 6
 2 8 7 3 5 4 9 1 4 1 9 2 6 8 5 3 7 2 5 3 4 9 7 1 6 8 8 7 6 3 2 1 9 4 5 9 4
 1 8 5 6 7 2 3]
582973614164582379397614852735149286628735491419268537253497168876321945941856723
Aver Time:   0.73 secs. Success rate: 860 / 860 
[7 6 4 8 1 5 9 2 3 8 1 5 2 3 9 6 4 7 2 9 3 4 6 7 1 8 5 4 5 6 1 7 2 8 3 9 3
 8 9 6 5 4 2 7 1 1 2 7 9 8 3 4 5 6 5 4 1 3 2 6 7 9 8 9 7 8 5 4 1 3 6 2 6 3
 2 7 9 8 5 1 4]
764815923815239647293467185456172839389654271127983456541326798978541362632798514
[3 5 8 2 9 1 6 4 7 7 4 6 3 8 5 2 9 1 1 2 9 6 4 7 3 8 5 4 1 2 5 6 9 7 3 8 9
 6 5 7 3 8 4 1 2 8 7 3 1 2 4 9 5 6 5 3 7 9 1 6 8 2 4 2 8 1 4 7 3 5 6 9 6 9
 4 8 5 2 1 7 3]
35829164774638529112964738541256973896573

[1 3 8 6 2 7 5 4 9 9 6 2 1 4 5 8 7 3 7 4 5 3 9 8 6 1 2 5 2 9 4 8 3 1 6 7 4
 7 3 9 1 6 2 5 8 6 8 1 7 5 2 3 9 4 2 1 7 8 6 9 4 3 5 8 9 6 5 3 4 7 2 1 3 5
 4 2 7 1 9 8 6]
138627549962145873745398612529483167473916258681752394217869435896534721354271986
[4 1 3 5 7 8 9 2 6 7 2 6 1 9 4 3 5 8 8 9 5 6 2 3 4 1 7 2 5 4 9 6 1 8 7 3 3
 8 7 4 5 2 1 6 9 9 6 1 3 8 7 5 4 2 6 3 8 2 1 5 7 9 4 1 4 9 7 3 6 2 8 5 5 7
 2 8 4 9 6 3 1]
413578926726194358895623417254961873387452169961387542638215794149736285572849631
[9 8 6 7 4 2 1 3 5 7 2 5 3 1 6 8 4 9 1 3 4 9 5 8 2 6 7 4 9 7 5 8 3 6 1 2 5
 6 3 2 9 1 7 8 4 2 1 8 6 7 4 9 5 3 6 5 1 4 2 7 3 9 8 3 4 2 8 6 9 5 7 1 8 7
 9 1 3 5 4 2 6]
986742135725316849134958267497583612563291784218674953651427398342869571879135426
[7 1 6 4 3 9 2 8 5 3 9 4 2 8 5 1 7 6 8 2 5 1 6 7 9 3 4 2 6 9 3 7 4 8 5 1 4
 7 3 8 5 1 6 2 9 1 5 8 9 2 6 7 4 3 5 4 2 6 9 8 3 1 7 6 8 7 5 1 3 4 9 2 9 3
 1 7 4 2 5 6 8]
716439285394285176825167934269374851473851629158926743542698317687513492931742568
[2 5 8 4

[9 2 1 6 4 8 5 7 3 8 4 6 3 5 7 1 9 2 3 7 5 2 9 1 8 6 4 5 9 3 8 7 4 6 2 1 1
 6 2 5 3 9 4 8 7 7 8 4 1 6 2 3 5 9 2 3 9 4 8 5 7 1 6 4 1 8 7 2 6 9 3 5 6 5
 7 9 1 3 2 4 8]
921648573846357192375291864593874621162539487784162359239485716418726935657913248
[9 8 6 7 4 2 1 3 5 7 2 5 3 1 6 8 4 9 1 3 4 9 5 8 2 6 7 4 9 7 5 8 3 6 1 2 5
 6 3 2 9 1 7 8 4 2 1 8 6 7 4 9 5 3 6 5 1 4 2 7 3 9 8 3 4 2 8 6 9 5 7 1 8 7
 9 1 3 5 4 2 6]
986742135725316849134958267497583612563291784218674953651427398342869571879135426
[4 9 3 5 2 1 7 8 6 1 7 6 3 4 8 5 9 2 5 8 2 7 6 9 3 1 4 3 2 7 8 5 4 1 6 9 9
 6 1 2 7 3 8 4 5 8 5 4 1 9 6 2 3 7 7 4 8 6 3 5 9 2 1 6 3 5 9 1 2 4 7 8 2 1
 9 4 8 7 6 5 3]
493521786176348592582769314327854169961273845854196237748635921635912478219487653
[4 5 1 8 7 9 6 2 3 2 8 9 5 3 6 7 4 1 6 3 7 4 1 2 8 9 5 9 2 8 3 5 7 1 6 4 1
 4 3 6 9 8 5 7 2 7 6 5 2 4 1 9 3 8 8 9 4 1 6 3 2 5 7 3 1 6 7 2 5 4 8 9 5 7
 2 9 8 4 3 1 6]
451879623289536741637412895928357164143698572765241938894163257316725489572984316
[9 5 3 2

[3 5 1 9 8 6 4 2 7 4 2 6 7 5 1 8 9 3 9 7 8 3 2 4 5 6 1 5 9 7 6 4 8 3 1 2 6
 8 4 1 3 2 9 7 5 1 3 2 5 9 7 6 4 8 7 4 3 2 6 5 1 8 9 2 6 9 8 1 3 7 5 4 8 1
 5 4 7 9 2 3 6]
351986427426751893978324561597648312684132975132597648743265189269813754815479236
[8 1 3 6 4 5 2 7 9 4 6 5 7 2 9 1 8 3 9 2 7 8 3 1 4 5 6 5 8 1 2 9 6 3 4 7 7
 3 2 4 5 8 9 6 1 6 9 4 3 1 7 5 2 8 3 4 8 1 6 2 7 9 5 1 5 6 9 7 4 8 3 2 2 7
 9 5 8 3 6 1 4]
813645279465729183927831456581296347732458961694317528348162795156974832279583614
[6 7 4 2 5 8 9 3 1 8 5 2 9 1 3 7 4 6 3 9 1 6 4 7 2 8 5 4 2 9 1 8 6 3 5 7 5
 3 7 4 9 2 1 6 8 1 6 8 3 7 5 4 2 9 9 8 3 5 2 1 6 7 4 2 1 5 7 6 4 8 9 3 7 4
 6 8 3 9 5 1 2]
674258931852913746391647285429186357537492168168375429983521674215764893746839512
Aver Time:   0.73 secs. Success rate: 960 / 960 
[8 5 4 1 2 6 9 3 7 7 3 2 4 9 5 1 8 6 9 1 6 7 8 3 4 5 2 1 4 8 3 5 7 6 2 9 2
 9 5 8 6 1 7 4 3 6 7 3 2 4 9 5 1 8 3 2 9 6 1 4 8 7 5 5 8 1 9 7 2 3 6 4 4 6
 7 5 3 8 2 9 1]
85412693773249518691678345214835762929586

[4 9 1 7 3 6 5 8 2 3 7 6 5 8 2 9 4 1 2 8 5 4 9 1 3 7 6 5 1 7 2 6 9 4 3 8 8
 3 4 1 5 7 6 2 9 9 6 2 3 4 8 7 1 5 1 4 9 8 7 5 2 6 3 6 2 3 9 1 4 8 5 7 7 5
 8 6 2 3 1 9 4]
491736582376582941285491376517269438834157629962348715149875263623914857758623194
[3 6 8 1 2 4 7 5 9 9 7 5 3 8 6 2 1 4 2 1 4 7 5 9 8 3 6 5 3 6 4 9 7 1 2 8 1
 4 9 2 3 8 6 7 5 8 2 7 5 6 1 4 9 3 7 5 3 8 4 2 9 6 1 4 9 2 6 1 5 3 8 7 6 8
 1 9 7 3 5 4 2]
368124759975386214214759836536497128149238675827561493753842961492615387681973542
[8 2 4 9 3 7 5 6 1 7 3 9 6 5 1 8 4 2 5 1 6 8 2 4 9 7 3 1 4 8 5 6 3 2 9 7 2
 7 3 1 4 9 6 8 5 6 9 5 2 7 8 3 1 4 4 6 7 3 8 2 1 5 9 9 8 2 7 1 5 4 3 6 3 5
 1 4 9 6 7 2 8]
824937561739651842516824973148563297273149685695278314467382159982715436351496728
[4 2 5 6 8 1 9 3 7 7 9 3 5 4 2 1 8 6 1 6 8 9 7 3 4 2 5 9 7 2 3 1 5 6 4 8 5
 3 1 8 6 4 2 7 9 8 4 6 7 2 9 3 5 1 2 8 9 1 3 7 5 6 4 3 1 7 4 5 6 8 9 2 6 5
 4 2 9 8 7 1 3]
425681937793542186168973425972315648531864279846729351289137564317456892654298713
[3 1 8 6

In [4]:
#     if np.linalg.norm(np.reshape(np.array([np.argmax(d)+1 for d in z]), (9,9) ) \
#                       - np.reshape([int(c) for c in solu], (9,9)), np.inf) >0:
        
        
#         L = 1
#         epsilon = 10**-10
#         for i in range(L):
#             # need W to be diagonal of size 9^3*2 x 9^3*2
            
#             x = np.absolute(x)
#             print(x)
#             x = x+ epsilon
#             print(x)
#             x = 1/x
#             print(x)
#             W = np.matrix(x).reshape(1458, 1458)
#             print(W)
            
        
  #      pass